# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
import numpy as np
import pandas as pd

from sqlalchemy import create_engine
import sqlite3

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import classification_report,accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///ETL.db')
df = pd.read_sql('SELECT * FROM ETL_Pipeline_Preparation', engine)

In [3]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df.isnull().sum()

id                            0
message                       0
original                  16046
genre                         0
related                       0
request                       0
offer                         0
aid_related                   0
medical_help                  0
medical_products              0
search_and_rescue             0
security                      0
military                      0
child_alone                   0
water                         0
food                          0
shelter                       0
clothing                      0
money                         0
missing_people                0
refugees                      0
death                         0
other_aid                     0
infrastructure_related        0
transport                     0
buildings                     0
electricity                   0
tools                         0
hospitals                     0
shops                         0
aid_centers                   0
other_in

In [5]:
df[df["original"].isnull()]

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
7423,8365,NOTES: It mark as not enough information,None,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9891,11186,My thoughts and prayers go out to all the live...,None,social,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
9892,11188,I m sorry for the poor people in Haiti tonight...,None,social,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
9893,11189,RT selenagomez UNICEF has just announced an em...,None,social,1,0,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0
9894,11192,lilithia yes 5.2 magnitude earthquake hit mani...,None,social,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
9895,11193,RT TheNewsBlotter RT caribnews On Call Interna...,None,social,1,0,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0
9896,11195,Most Eureka homeowners won&#39 t collect on qu...,None,social,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
9897,11196,Haiti hit by largest earthquake in over 200 ye...,None,social,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
9898,11197,RT pinkelephantpun Earthquake Relief Donate ht...,None,social,1,1,0,1,0,0,...,0,0,1,0,0,0,1,0,0,0
9899,11198,Praying that today s earthquake in #Haiti mini...,None,social,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0


In [6]:
X=df["message"]
y= df.drop(columns=["id","message", "original", "genre"])

### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    """ 
    tokenization and lemmatization of text after removing punctiations
  
    Parameters: 
    text (str): text data 
  
    Returns: 
    list: list of strings 
  
    """
    # removing punctiations and returning lower case
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    #tokenization
    tokens = word_tokenize(text)
    
    #lemmatization
    lemmatizer = WordNetLemmatizer()
    
    return [lemmatizer.lemmatize(token).lower().strip() for token in tokens]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [12]:
y_pred = pipeline.predict(X_test)

In [13]:
av_accuracy=(y_pred == y_test).mean().mean()
print("Avarage accuracy: ", round(av_accuracy*100,0),"%")

Avarage accuracy:  94.0 %


In [14]:
msg = ['I am starving']
test_output = pipeline.predict(msg)
print(y_train.columns.values[(test_output.flatten()==1)])

['related' 'request' 'direct_report']


In [16]:
msg = ['Hello I see fire in the street and many houses are destroyed, homeless people everywhere']
test_output = pipeline.predict(msg)
print(y_train.columns.values[(test_output.flatten()==1)])

['related' 'aid_related' 'weather_related']


In [17]:
for i in range(y.shape[1]):
    print(y.columns.values[i],'\n', classification_report(y_test.values[:,i], y_pred[:,i]))

related 
              precision    recall  f1-score   support

          0       0.63      0.35      0.45      1550
          1       0.82      0.94      0.87      4951
          2       0.58      0.13      0.22        53

avg / total       0.77      0.79      0.77      6554

request 
              precision    recall  f1-score   support

          0       0.89      0.98      0.93      5415
          1       0.84      0.40      0.54      1139

avg / total       0.88      0.88      0.86      6554

offer 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6528
          1       0.00      0.00      0.00        26

avg / total       0.99      1.00      0.99      6554

aid_related 
              precision    recall  f1-score   support

          0       0.72      0.88      0.80      3815
          1       0.77      0.53      0.62      2739

avg / total       0.74      0.74      0.72      6554

medical_help 
              precision    reca

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
parameters = {
    "tfidf__use_idf": (True, False),              
}

cv = GridSearchCV(pipeline, parameters, cv=2, verbose=3)

In [13]:
type(cv)

sklearn.model_selection._search.GridSearchCV

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [19]:
cv.fit(X_train, y_train)

GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'tfidf__use_idf': (True, False)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [33]:
model=cv.best_estimator_
y_pred=model.predict(X_test)

In [34]:
cv.best_params_

{'tfidf__use_idf': True}

In [35]:
av_accuracy=(y_pred == y_test).mean().mean()
print("Avarage accuracy: ", round(av_accuracy*100,0),"%")

Avarage accuracy:  94.0 %


In [36]:
msg = ['Hello I see fire in the street and many houses are destroyed, homeless people everywhere']
test_output = model.predict(msg)
print(y_train.columns.values[(test_output.flatten()==1)])

['related' 'aid_related']


In [37]:
msg = ['I am starving']
test_output = model.predict(msg)
print(y_train.columns.values[(test_output.flatten()==1)])

['related' 'request' 'food']


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(KNeighborsClassifier()))
    ])

### 9. Export your model as a pickle file

In [48]:
import pickle
filename = 'ML_pipline_final.pkl'
pickle.dump(model, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.